In [ ]:
from ipycanvas import Canvas, hold_canvas
from IPython.display import Image, clear_output
import random

FARBEN = '♥♠♦♣'
WERTE = ('2', '3', '4', '5', '6', '7', '8', '9', '10', 'B', 'D', 'K', 'A')

def create_deck():
    deck = [wert + farbe for farbe in FARBEN for wert in WERTE]
    random.shuffle(deck)
    return deck

def get_card_value(card):
    wert = card[:-1]
    if wert.isdigit():
        return int(wert)
    elif wert in ('B', 'D', 'K'):
        return 10
    else:
        return 'A'

def calculate_hand_value(hand):
    total = 0
    num_aces = 0
    
    for card in hand:
        value = get_card_value(card)
        if value == 'A':
            num_aces += 1
        else:
            total += value
    
    for _ in range(num_aces):
        if total + 11 <= 21:
            total += 11
        else:
            total += 1
    
    return total

def draw_card(canvas, card, x, y, hide=False): 
    width = 60 
    height = 100 
    border_width = 2 

    canvas.fill_style = 'white' 
    canvas.fill_rect(x, y, width, height) 

    if not hide:
        canvas.fill_style = 'red' if card[-1] in ['♥', '♦'] else 'black' 
        canvas.font = 'bold 24px sans-serif' 
        canvas.fill_text(card[-1], x + 10, y + 40)

        canvas.font = 'bold 20px sans-serif'
        value = 'A' if card[:-1] == '1' else card[:-1]
        canvas.fill_text(value, x + width // 2 - 8, y + height - 20)

        canvas.fill_style = 'black'
        canvas.fill_rect(x + width, y, border_width, height)

def display_hand(canvas, hand, x, y, hide_first_card=False, label="Spieler", player_name=None):
    for i, card in enumerate(hand):
        if hide_first_card and i == 0: 
            draw_card(canvas, "", x + i * (60 + 10), y, hide=True)
        else:
            draw_card(canvas, card, x + i * (60 + 10), y)

    # Label anzeigen
    canvas.fill_style = 'black'
    canvas.font = 'bold 20px Verdana, sans-serif'
    if player_name:
        canvas.fill_text(player_name, x, y - 5)
    else:
        canvas.fill_text(label, x, y - 5)

def display_money(canvas, money):
    money_str = f"Dein Betrag: {int(money)} Franken"
    canvas.fill_style = 'black'
    canvas.font = 'bold 20px Verdana, sans-serif'
    canvas.fill_text(money_str, 20, 200)

def display_canvas(canvas):
    display(canvas)
    canvas.fill_style = "#008b00"
    canvas.fill_rect(0, 0, canvas.width, canvas.height)

def start_new_game(deck, player_hand, dealer_hand, canvas, player_money, player_name, character_choice):
    if len(deck) < 52:
        deck = create_deck()

    player_hand.clear()
    dealer_hand.clear()
    
    player_hand.extend([deck.pop(), deck.pop()])
    dealer_hand.extend([deck.pop(), deck.pop()])

    with hold_canvas(canvas):
        canvas.clear()
        canvas.fill_style = "#008b00"
        canvas.fill_rect(0, 0, canvas.width, canvas.height)

        display_hand(canvas, player_hand, 20, 40, label="Spieler", player_name=player_name)  #zeigt die Hand des Spielers auf dem Canvas an
        display_hand(canvas, dealer_hand, 700, 40, hide_first_card=True, label="Dealer") #zeigt die Hand des Dealers auf dem Canvas an
        display_money(canvas, player_money) #zeigt den Geldbetrag des Spielers auf dem Canvas an
        display_character_image(character_choice) #zeigt das Bild des ausgewählten Charakters auf dem Canvas an

    if len(player_hand) == 2 and player_hand[0][:-1] == player_hand[1][:-1]:
        split_choice = input("Möchtest du teilen? (j/n): ").lower()
        if split_choice == 'j':
            # Split the hand
            player_hand_1 = [player_hand[0], deck.pop()]
            player_hand_2 = [player_hand[1], deck.pop()]

            with hold_canvas(canvas):
                canvas.clear()
                canvas.fill_style = "#008b00"
                canvas.fill_rect(0, 0, canvas.width, canvas.height)

                display_hand(canvas, player_hand_1, 20, 40, label="Spieler Hand 1", player_name=player_name)
                display_hand(canvas, player_hand_2, 300, 40, label="Spieler Hand 2", player_name=player_name)
                display_hand(canvas, dealer_hand, 700, 40, hide_first_card=True, label="Dealer")
                display_money(canvas, player_money)
                display_character_image(character_choice)

            return [player_hand_1, player_hand_2], dealer_hand

    return [player_hand], dealer_hand

def display_character_image(character_choice):
    if character_choice == '1':
        image_path = "Roboter_1.png"
    elif character_choice == '2':
        image_path = "Mario_1.png"
    elif character_choice == '3':
        image_path = "Joker_1.png"
    else:
        print("Ungültige Charakterwahl.")
        return
    
    display(Image(filename=image_path))


def main():
    print("\033[1m\033[25mWillkommen bei Blackjack!\033[0m")
    player_name = input("Gib deinen Namen ein: ")
    print("Wähle deinen Charakter:")
    print("1. Roboter")
    print("2. Mario")
    print("3. Joker")
    character_choice = input("Welchen Charakter möchtest du spielen? (1-3): ")
    
    if character_choice not in ('1', '2', '3'):
        print("Ungültige Eingabe. Wähle einen Charakter von 1 bis 3.")
        return

    player_money = 1000

    deck = create_deck()
    player_hand = []
    dealer_hand = []
    
    while player_money > 0:
        canvas = Canvas(width=1000, height=230)
        display_canvas(canvas)
        
        print("\n\033[1mNeue Runde beginnt!\033[0m")
        print(f"{player_name}, du hast {int(player_money)} Franken.")
        while True:
            try:
                bet = input("Wie viel möchtest du setzen? ")
                if not bet.replace('.', '').isdigit():  
                    raise ValueError("Bitte geben Sie einen numerischen Betrag ein.")
                bet = float(bet)
                if bet < 5:
                    raise ValueError("Der Mindesteinsatz beträgt 5 Franken.")
                if bet != int(bet):
                    raise ValueError("Es sind keine Komma-Beträge erlaubt.")
                if bet > player_money:
                    raise ValueError("Du hast nicht genug Geld!")
                player_money -= bet
                break
            except ValueError as e:
                print(e)

        hands, dealer_hand = start_new_game(deck, player_hand, dealer_hand, canvas, player_money, player_name, character_choice)

        for hand in hands:
            while calculate_hand_value(hand) < 21:
                choice = input("Willst du eine Karte ziehen? (j/n): ").lower()
                if choice != 'j':
                    break
                hand.append(deck.pop())

                with hold_canvas(canvas):
                    canvas.clear()
                    canvas.fill_style = "#008b00"
                    canvas.fill_rect(0, 0, canvas.width, canvas.height)
                    for i, h in enumerate(hands):
                        display_hand(canvas, h, 20 + i * 280, 40, label=f"Spieler (Hand {i + 1})", player_name=player_name)
                    display_hand(canvas, dealer_hand, 700, 40, hide_first_card=True, label="Dealer")
                    display_money(canvas, player_money)

            player_score = calculate_hand_value(hand)
            if player_score <= 21:
                while calculate_hand_value(dealer_hand) < 17:
                    dealer_hand.append(deck.pop())

                with hold_canvas(canvas):
                    canvas.clear()
                    canvas.fill_style = "#008b00"
                    canvas.fill_rect(0, 0, canvas.width, canvas.height)
                    for i, h in enumerate(hands):
                        display_hand(canvas, h, 20 + i * 280, 40, label=f"Spieler (Hand {i + 1})", player_name=player_name)
                    display_hand(canvas, dealer_hand, 700, 40, label="Dealer")
                    display_money(canvas, player_money)

                dealer_score = calculate_hand_value(dealer_hand)
                if dealer_score > 21 or player_score > dealer_score:
                    player_money += bet * 2
                    print("\033[1m\033[33mHerzlichen Glückwunsch! Du gewinnst!\033[0m")
                elif player_score < dealer_score:
                    print("\033[1m\033[33mDealer gewinnt!\033[0m")
                else:
                    player_money += bet
                    print("\033[1m\033[33mUnentschieden!\033[0m")
            else:
                print("\033[1m\033[33mDu hast verloren!\033[0m")
        
        input("\nDrücken Sie Enter für die nächste Runde...")
        clear_output()

    print("\033[1m\033[33mDu hast kein Geld mehr. Spiel beendet!\033[0m")

main()